In [4]:
#eda stacks
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import s3fs
import os
import sys

#display settings
pd.set_option("display.max_columns", 200)      # show all columns
pd.set_option("display.max_rows", 20)         # show more rows
pd.set_option("display.width", 1200)           # wider output
pd.set_option("display.max_colwidth", 100)     # long text columns
pd.set_option("display.float_format", "{:.2f}".format)  # clean decimals
sns.set_theme(style="whitegrid")

In [5]:
bucket="kerala-ayurveda-s3"
prefix="gold-tier/funnel.csv"
s3_path=f"s3://{bucket}/{prefix}"
funnel=pd.read_csv(s3_path)

In [6]:
bucket="kerala-ayurveda-s3"
prefix="gold-tier/payments.csv"
s3_path=f"s3://{bucket}/{prefix}"
payments=pd.read_csv(s3_path)

In [15]:
funnel['date']=pd.to_datetime(funnel['date_ist']).dt.date
payments['date']=pd.to_datetime(payments['date_ist']).dt.date

In [16]:
funnel['week']=pd.to_datetime(funnel['date_ist']).dt.day.sub(1) // 7 + 1
payments['week']=pd.to_datetime(payments['date_ist']).dt.day.sub(1) // 7 + 1

In [17]:
funnel.sample()

,date_ist,device,landing_page_type,source,medium,campaign,users,sessions,product_view,add_to_cart,begin_checkout,add_payment_info,add_shipping_info,purchase,date,week
6481,2025-12-22,mobile,cart_page,facebook,cpc,Meta_Ads_KAL_IN_TOF_Conv_Sale_Feed_22112025_BreathEasy_Scale,1,1,1,1,1,0,1,0,2025-12-22,4


In [20]:
f=funnel.groupby(['week']).agg(
    users=('users','sum'),
    sessions=('sessions','sum'),
    prod_view=('product_view','sum'),
    atc=('add_to_cart','sum'),
    checkout=('begin_checkout','sum'),
    pay_info=('add_payment_info','sum'),
    ship_info=('add_shipping_info','sum'),
    purchase=('purchase','sum'),
).reset_index()

In [21]:
f['cvr']=f['purchase']*100.0/f['users']

In [22]:
f

,week,users,sessions,prod_view,atc,checkout,pay_info,ship_info,purchase,cvr
0,1,78780,81044,80470,8164,3989,2247,3127,1753,2.23
1,2,70376,72302,71678,7227,3614,1968,2815,1512,2.15
2,3,63308,65117,64511,7048,3663,2088,2855,1605,2.54
3,4,66553,68544,67933,7233,3918,2166,3042,1638,2.46
4,5,28804,29830,29539,3110,1681,929,1323,710,2.46


In [27]:
p=payments.groupby(['week']).agg(
    users=('users','sum'),
    sessions=('sessions','sum'),
    payers=('payers','sum'),
    orders=('orders','sum'),
    rev=('rev','sum')
).reset_index()

In [28]:
p['aov']=p['rev']/p['orders']
p['arppu']=p['rev']/p['payers']
p['arpu']=p['rev']/p['users']

In [29]:
p

,week,users,sessions,payers,orders,rev,aov,arppu,arpu
0,1,78780,81044,1752,1784.00,1048852.43,587.92,598.66,13.31
1,2,70376,72302,1512,1534.00,941594.22,613.82,622.75,13.38
2,3,63308,65117,1605,1624.00,933388.15,574.75,581.55,14.74
3,4,66553,68544,1637,1663.00,998867.50,600.64,610.18,15.01
4,5,28804,29830,708,718.00,437774.80,609.71,618.33,15.20


Week 1 vs Week 4

In [30]:
funnel.sample()

,date_ist,device,landing_page_type,source,medium,campaign,users,sessions,product_view,add_to_cart,begin_checkout,add_payment_info,add_shipping_info,purchase,date,week
6573,2025-12-22,mobile,products,bing,organic,(organic),1,1,1,0,0,0,0,0,2025-12-22,4


In [44]:
week_funnel=funnel[funnel['week'].isin([1,4])]
comp=week_funnel.groupby(['week','campaign']).agg(
    Users=('users','sum')
).reset_index()
comp.sort_values(by=['week','Users'],ascending=[True,False],inplace=True)

In [46]:
# comp[comp['source'].isin(['google','facebook','wishlink','skendra','Limechat'])]
# comp[comp['medium'].isin(['cpc','organic','wishlink.com','affiliate','Internal Channel'])]
comp['rank'] = (
    comp.groupby('week')['Users']
      .rank(method='dense', ascending=False)
)
comp[comp['rank']<=5]


,week,campaign,Users,rank
226,1,Google_Shopping_Selected_Products_MC_12082025,14985,1.00
0,1,(organic),9627,2.00
228,1,Google_Shopping_Selected_Products_TROAS_Cities_20082025,8173,3.00
218,1,Google_PMax_Selected_Products_12072025Google_PMax_Selected_Products_MC_28112025_Scale,7154,4.00
230,1,Google_Shopping_Skin_and_Hair_Category_MC_07092025,6114,5.00
875,4,Google_Shopping_Selected_Products_MC_12082025,14334,1.00
866,4,Google_PMax_Selected_Products_12072025Google_PMax_Selected_Products_MC_28112025_Scale,7243,2.00
877,4,Google_Shopping_Selected_Products_TROAS_Cities_20082025,7219,3.00
868,4,Google_Pmax_Rem_Allproducts_Pan_211024,6065,4.00
891,4,IND_ACQ_Web_Google_PMax_Products_Purchases_CList_India,4775,5.00


In [48]:
one=comp[comp['week']==1]
four=comp[comp['week']==4]
campaign_check=one.merge(four,how='inner',on='campaign')
campaign_check.sample()

,week_x,campaign,Users_x,rank_x,week_y,Users_y,rank_y
70,1,agent_sale_dr_harsha,1,45.00,4,1,53.00


In [49]:
campaign_check.rename(columns={'Users_x':'Users_Week1','Users_y':'Users_Week4','rank_x':'Rank_Week1','rank_y':'Rank_Week4'},inplace=True)
campaign_check.sample()

,week_x,campaign,Users_Week1,Rank_Week1,week_y,Users_Week4,Rank_Week4
38,1,14Nov_champions_loyal__active,8,38.00,4,2,52.00


In [53]:
campaign_check.sort_values(by='Users_Week1',ascending=False,inplace=True)
campaign_check.drop(columns=['week_x','week_y']).head(10)

,campaign,Users_Week1,Rank_Week1,Users_Week4,Rank_Week4
0,Google_Shopping_Selected_Products_MC_12082025,14985,1.00,14334,1.00
1,(organic),9627,2.00,1117,12.00
2,Google_Shopping_Selected_Products_TROAS_Cities_20082025,8173,3.00,7219,3.00
3,Google_PMax_Selected_Products_12072025Google_PMax_Selected_Products_MC_28112025_Scale,7154,4.00,7243,2.00
4,Google_Shopping_Skin_and_Hair_Category_MC_07092025,6114,5.00,4003,8.00
5,Google_Pmax_Rem_Allproducts_Pan_211024,5664,6.00,6065,4.00
6,Google_Shopping_Selected_Products_MC_28112025_Scale,4890,7.00,4247,7.00
7,Meta_Ads_KAL_IN_TOF_Conv_ASC_DPA_29092025,3832,8.00,4596,6.00
8,IND_ACQ_Web_Google_PMax_Products_Purchases_CList_India,3190,9.00,4775,5.00
9,Meta_Ads_KAL_IN_BOF_Conv_BAU_Feed_17072025_Remarketing,2967,10.00,16,41.00


In [54]:
week_funnel=payments[payments['week'].isin([1,4])]
comp=week_funnel.groupby(['week','campaign']).agg(
    Payers=('payers','sum')
).reset_index()
comp.sort_values(by=['week','Payers'],ascending=[True,False],inplace=True)
# comp[comp['source'].isin(['google','facebook','wishlink','skendra','Limechat'])]
# comp[comp['medium'].isin(['cpc','organic','wishlink.com','affiliate','Internal Channel'])]
comp['rank'] = (
    comp.groupby('week')['Payers']
      .rank(method='dense', ascending=False)
)
comp[comp['rank']<=5]

one=comp[comp['week']==1]
four=comp[comp['week']==4]
campaign_check=one.merge(four,how='inner',on='campaign')
campaign_check.sample()
campaign_check.rename(columns={'Payers_x':'Payers_Week1','Payers_y':'Payers_Week4','rank_x':'Rank_Week1','rank_y':'Rank_Week4'},inplace=True)
campaign_check.sample()
campaign_check.sort_values(by='Payers_Week1',ascending=False,inplace=True)
campaign_check.drop(columns=['week_x','week_y']).head(10)

,campaign,Payers_Week1,Rank_Week1,Payers_Week4,Rank_Week4
0,Google_Shopping_Selected_Products_MC_12082025,282,1.00,227,1.00
1,Google_Shopping_Selected_Products_TROAS_Cities_20082025,201,2.00,154,2.00
2,(organic),163,3.00,49,12.00
3,Google_PMax_Selected_Products_12072025Google_PMax_Selected_Products_MC_28112025_Scale,147,4.00,128,3.00
4,Google_Shopping_Skin_and_Hair_Category_MC_07092025,115,5.00,89,7.00
5,Google_Shopping_Selected_Products_MC_28112025_Scale,89,6.00,108,5.00
6,Google_Pmax_Brand_Allproducts_Pan_20082025,85,7.00,122,4.00
7,nov25,78,8.00,70,10.00
8,Meta_Ads_KAL_IN_TOF_Conv_ASC_DPA_29092025,73,9.00,107,6.00
9,IND_ACQ_Web_Google_PMax_Products_Purchases_CList_India,67,10.00,81,9.00
